In [2]:
println!("HELLO WORLD")

HELLO WORLD


()

In [14]:
:dep sophia = { version = "0.7.2" }
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = { version = "1.0" }
:dep weak-table = { version = "0.3.2" }

extern crate sophia;
extern crate serde;
extern crate serde_json;
extern crate weak_table;

## Calculate diff between triples

In [4]:
use sophia::parser::turtle;
use sophia::term::TTerm;
use sophia::triple::stream::TripleSource;
use sophia::triple::Triple;

use std::collections::HashSet;


fn test() {
    let example = r#"
        @prefix : <http://example.org/>.
        @prefix foaf: <http://xmlns.com/foaf/0.1/>.

        :alice foaf:name "Alice";
               foaf:mbox <mailto:alice@work.example> .

        :bob foaf:name "Bob".
    "#;

    let mut set: HashSet<[String; 3]> = HashSet::<[String; 3]>::new();
    turtle::parse_str(example).for_each_triple(|t| {
        set.insert([String::from(t.s().value()), String::from(t.p().value()), String::from(t.o().value())]);
    });

    let example2 = r#"
        @prefix : <http://example.org/>.
        @prefix foaf: <http://xmlns.com/foaf/0.1/>.

        :alice foaf:name "Alice";
               foaf:mbox <mailto:alice@work.example> .

        :carol foaf:name "Carol".
    "#;

    let mut set2: HashSet<[String; 3]> = HashSet::<[String; 3]>::new();
    turtle::parse_str(example2).for_each_triple(|t| {
        set2.insert([String::from(t.s().value()), String::from(t.p().value()), String::from(t.o().value())]);
    });


    // Set difference -> elms in set but not in set2 (can be seen as 'additions')
    println!("\nElements in set but not in set2:");
    for x in set.difference(&set2) {
        println!("{:?}", x);
    }

    // Set difference -> elms in set2 but not in set (can be seen as 'additions')
    println!("\nElements in set2 but not in set:");
    for x in set2.difference(&set) {
        println!("{:?}", x);
    }
}

test();


Elements in set but not in set2:
["http://example.org/bob", "http://xmlns.com/foaf/0.1/name", "Bob"]

Elements in set2 but not in set:
["http://example.org/carol", "http://xmlns.com/foaf/0.1/name", "Carol"]


In [ ]:
fn get_diff() -> LightGraph, LightGraph {
    
}

## Convert wikidata item json to RDF

In [72]:
use serde::{Serialize, Deserialize};

use std::collections::HashMap;
use std::vec::Vec;


#[derive(Debug, Clone, Serialize, Deserialize)]
struct WikidataItem {
    id: String,
    
    #[serde(rename="type")]
    wd_type: String,
    
    labels: HashMap<String, TextValue>,
    
    descriptions: HashMap<String, TextValue>,
    
    aliases: HashMap<String, Vec<TextValue>>,
    
    claims: HashMap<String, Vec<Statement>>,
    
    sitelinks: HashMap<String, Sitelink>,
    
    lastrevid: u32,
    
    modified: String
}

#[derive(Debug, Clone, Serialize, Deserialize)]
struct TextValue {
    language: String,
    value: String
}

#[derive(Debug, Clone, Serialize, Deserialize)]
struct Statement {
    id: String,
    
    mainsnak: Snak,
    
    #[serde(rename="type")]
    st_type: String,
    
    rank: String,
    
    qualifiers: Option<HashMap<String, Vec<Snak>>>,
    
    references: Option<Vec<Reference>>
}

#[derive(Debug, Clone, Serialize, Deserialize)]
struct Snak {
    hash: Option<String>,
    
    snaktype: SnakType,
    
    property: String,
    
    datatype: String,
    
    datavalue: Option<DataValue>
}

#[derive(Debug, Clone, Serialize, Deserialize)]
struct Reference {
    hash: String,
    
    snaks: HashMap<String, Vec<Snak>>,
    
    #[serde(rename="snaks-order")]
    snaks_order: Vec<String>
}

#[derive(Debug, Clone, Serialize, Deserialize)]
struct Sitelink {
    site: String,
    
    title: String,
    
    badges: Vec<String>,
    
    url: Option<String>
}

#[derive(Debug, Clone, Serialize, Deserialize)]
#[serde(rename_all="snake_case")]
enum SnakType {
    Value,
    Somevalue,
    Novalue
}

#[derive(Debug, Clone, Serialize, Deserialize)]
struct WikibaseEntityId {
    #[serde(rename="entity-type")]
    entity_type: String,

    id: String,
    
    #[serde(rename="numeric-id")]
    numeric_id: Option<u32>
}

#[derive(Debug, Clone, Serialize, Deserialize)]
struct GlobeCoordinate {
    latitude: f64,
    
    longitude: f64,
    
    precision: f64,
    
    globe: String
}

#[derive(Debug, Clone, Serialize, Deserialize)]
struct Quantity {
    amount: String,
    
    upperBound: Option<String>,
    
    lowerBound: Option<String>,
    
    unit: String
}


#[derive(Debug, Clone, Serialize, Deserialize)]
struct Time {
    time: String,
    
    timezone: i16,
    
    calendarmodel: String,
    
    precision: u8,
    
    before: u8,
    
    after: u8
}


#[derive(Debug, Clone, Serialize, Deserialize)]
struct MonolingualText {
    text: String,
    language: String
}

#[derive(Debug, Clone, Serialize, Deserialize)]
#[serde(tag="type", content="value")]
enum DataValue {
    #[serde(rename="string")]
    ValueStr(String),
    
    #[serde(rename="wikibase-entityid")]
    WikibaseEntityID(WikibaseEntityId),
    
    #[serde(rename="globecoordinate")]
    ValueGlobeCoordinate(GlobeCoordinate),
    
    #[serde(rename="quantity")]
    ValueQuantity(Quantity),
    
    #[serde(rename="time")]
    ValueTime(Time),
    
    #[serde(rename="monolingualtext")]
    ValueMonolingualText(MonolingualText)
}

In [73]:
use serde::Deserialize;

use std::fs::File;
use std::io::BufReader;
use std::path::Path;


let path = "Q42.json";

let file = File::open(path).unwrap();
let reader = BufReader::new(file);

// Read the JSON contents of the file as an instance of `WikidataItem`.
let wd_item: WikidataItem = serde_json::from_reader::<_, WikidataItem>(reader).unwrap();

In [ ]:
fn wd_json_to_rdf(item: WikidataItem) -> LightGraph {
    
}

In [70]:
use sophia::graph::inmem::{HashGraph, LightGraph, TermIndexMapU};
use sophia::graph::MutableGraph;
use sophia::iri::Iri;
use sophia::ns::Namespace;
use sophia::prefix::{Prefix, PrefixMap};
use sophia::serializer::turtle::{TurtleConfig, TurtleSerializer};
use sophia::serializer::{Stringifier, TripleSerializer};

use std::error::Error;

fn do_stuff() {
    let mut graph = LightGraph::new();
    let ex = Namespace::new("http://example.org/").unwrap();
    let foaf = Namespace::new("http://xmlns.com/foaf/0.1/").unwrap();
    let prefix_map: Vec<(Prefix<'static>, Iri<'static>)> = vec![
            (
                Prefix::new_unchecked("ex"),
                Iri::new_unchecked("http://example.org/"),
            ),
            (
                Prefix::new_unchecked("foaf"),
                Iri::new_unchecked("http://xmlns.com/foaf/0.1/"),
            )
        ];
    
    graph.insert(
        &ex.get("bob").unwrap(),
        &foaf.get("knows").unwrap(),
        &ex.get("alice").unwrap(),
    ).unwrap();

    let mut config = TurtleConfig::new().with_pretty(true).with_prefix_map(&prefix_map[..]);
    let mut ttl_stringifier = TurtleSerializer::new_stringifier_with_config(config);
    let example2 = ttl_stringifier.serialize_graph(&mut graph).unwrap().as_str();
    println!("The resulting graph\n{}", example2);
}

In [71]:
do_stuff();

The resulting graph
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

ex:bob 
  foaf:knows ex:alice.


